# Search embedding

Virtual environment
https://www.geeksforgeeks.org/using-jupyter-notebook-in-virtual-environment/

In [1]:
# !pip install -U sentence-transformers

## Load dataset

In [2]:
SEARCH_DATA_PATH = ...
PROCESSED_DATA_PATH = "../datasets/processed_datasets"
DATA_INDEX_PATH = "../datasets/"
EMBEDDING_DATA_PATH = "../datasets/search_embeddings"
MODEL_PATH = "../models/"

SEARCH_DATA_PATH = "../datasets/processed_datasets"
SEARCH_DATA_INDEX_PATH = "../datasets/raw_datasets"

In [3]:
# import pandas as pd
# dataset_train = pd.read_json(PROCESSED_DATA_PATH+"/dataset_spacesplit_train.json")
# dataset_test = pd.read_json(PROCESSED_DATA_PATH+"/dataset_spacesplit_test.json")
# data_index = pd.read_csv(DATA_INDEX_PATH+'/data_index.csv')

# dataset_train["text_str"] = [" ".join(v) for v in dataset_train["text"]]
# dataset_test["text_str"] = [" ".join(v) for v in dataset_test["text"]]

#### load search data

In [4]:
import pandas as pd
data_vote = pd.read_json(SEARCH_DATA_PATH+"/search_dataset_vote.json")
data_active = pd.read_json(SEARCH_DATA_PATH+"/search_dataset_active.json")
data_hot = pd.read_json(SEARCH_DATA_PATH+"/search_dataset_hot.json")

search_data_index_hot = pd.read_csv(SEARCH_DATA_INDEX_PATH+"/metadata-for-search-info-hottest-rating.csv")
search_data_index_vote = pd.read_csv(SEARCH_DATA_INDEX_PATH+"/metadata-for-search-info.csv")
search_data_index_active = pd.read_csv(SEARCH_DATA_INDEX_PATH+"/metadata-for-search-info-active-rating.csv")

In [5]:
data_hot['id'], data_hot['text']

(0        0
 1        1
 2        2
 3        3
 4        4
       ... 
 684    685
 685    686
 686    687
 687    688
 688    689
 Name: id, Length: 689, dtype: int64,
 0       Top 10000 Popular Movies Dataset Top 10000 Po...
 1       IMDB Top 1000 Movies Dataset "Release Year, D...
 2       Top 1000 IMDb Movies Dataset Discover the Gre...
 3       Top Rated 10000 Movies Dataset (IMDB) In this...
 4       Top 1000 Highest Grossing Movies Top 1000 Hig...
                              ...                        
 684     Scene Classification Contains ~25K images fro...
 685     Plants Classification 30 Types of Plants Imag...
 686     Flower Classification 14 Types of Flower Imag...
 687     BIRDS 525  SPECIES- IMAGE CLASSIFICATION 525 ...
 688     Brain Tumor Classification (MRI) Classify MRI...
 Name: text, Length: 689, dtype: object)

In [6]:
queries = search_data_index_vote['query name'].unique()

## Inference batching utils

In [7]:
def iterate_batch(dataset,batch_size=128):
    for i in range(0,len(dataset),batch_size):
        yield dataset[i:i+batch_size]

In [8]:
iterate_batch([1,2,3,4,5,6,7,8,9,10],3)

<generator object iterate_batch at 0x7f2b72a22880>

## Models

### Standart models

https://huggingface.co/Muennighoff/SGPT-2.7B-weightedmean-msmarco-specb-bitfit
BERT
https://huggingface.co/princeton-nlp/sup-simcse-bert-base-uncased

In [13]:
import gc

#### top 1 Salesforce/SFR-Embedding-2_R

In [14]:
import torch
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel

tokenizer_SFR = AutoTokenizer.from_pretrained('Salesforce/SFR-Embedding-2_R',cache_dir=MODEL_PATH)
model_SFR = AutoModel.from_pretrained('Salesforce/SFR-Embedding-2_R',cache_dir=MODEL_PATH)

def sfr_model_inference(text=None,dataset=None):
    global tokenizer_SFR
    global model_SFR
    def last_token_pool(last_hidden_states: Tensor,
                     attention_mask: Tensor) -> Tensor:
        left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
        if left_padding:
            return last_hidden_states[:, -1]
        else:
            sequence_lengths = attention_mask.sum(dim=1) - 1
            batch_size = last_hidden_states.shape[0]
            return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]
    
    def get_detailed_instruct(task_description: str, query: str) -> str:
        return f'Instruct: {task_description}\nQuery: {query}'

    # Query process
    # Each query must come with a one-sentence instruction that describes the task
    # task = 'Given a web search query, retrieve relevant passages that answer the query'
    # queries = [
    #     get_detailed_instruct(task, '<Query1>'),
    #     get_detailed_instruct(task, '<Query2>')
    # ]

    with torch.no_grad():
        # load model and tokenizer
        tokenizer = tokenizer_SFR
        model = model_SFR
        model = model.to(device)
        embeddings = 0
        model.eval()

        if text is None:
            text = list(dataset["text_str"])
        for batch in iterate_batch(text,10):
            # get the embeddings
            embedding=0
            try:
                model = model.to(device)
                max_length = 4096
                batch_dict = tokenizer(batch, max_length=max_length, padding=True, truncation=True, return_tensors="pt").to(device)
                outputs = model(**batch_dict)
                embedding = last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
            
                # normalize embeddings
                embedding = F.normalize(embedding, p=2, dim=1)
                embedding = embedding.to('cpu')
    
                if type(embeddings) is int:
                    embeddings = embedding
                else:
                    embeddings = torch.cat((embeddings,embedding),dim=0)
            except torch.cuda.OutOfMemoryError:
                del model
                del batch_dict
                del embedding
                torch.cuda.empty_cache()
                raise torch.cuda.OutOfMemoryError
            del embedding
            del batch_dict
            torch.cuda.empty_cache()
        del model
        del tokenizer
        torch.cuda.empty_cache()
        return embeddings

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Loading checkpoint shards: 100%|██████████| 3/3 [00:10<00:00,  3.37s/it]


In [283]:
model_name = "SFR"
dataset = dataset_train

embeddings = sfr_model_inference(dataset)
evaluate(embeddings,model_name)
store_vectors(embeddings,dataset,model_name+"_inference")

del embeddings
torch.cuda.empty_cache()
gc.collect()

NameError: name 'dataset_train' is not defined

In [ ]:
pair_vise_score = (embeddings@embeddings.T)

In [ ]:
pair_vise_score

BERT

In [ ]:
https://huggingface.co/princeton-nlp/sup-simcse-bert-base-uncased

LLAMA
https://huggingface.co/McGill-NLP/LLM2Vec-Meta-Llama-3-8B-Instruct-mntp-supervised

### Load vector database

In [20]:
all_results_hot = []
all_results_vote = []
all_results_active = []

In [21]:
metrics = {}

#### with no model

In [14]:
load_time = 0
search_time = 0

In [19]:
import chromadb
import time

metrics['default'] = {"active":{},"vote":{},"hot":{}}

client = chromadb.Client()

collection_vote = client.create_collection("seach_db_vote",get_or_create=False)
collection_active = client.create_collection("seach_db_active",get_or_create=False)
collection_hot = client.create_collection("seach_db_hot",get_or_create=False)
# collection = client.create_collection("test_seach_fb",embedding_function,get_or_create=True)

# load
load_time = time.time()
collection_vote.add(
    documents=list(data_vote['text']),
    ids=[str(i) for i in data_vote['id']]
)
load_time = time.time() - load_time
metrics['default']['vote']['load_time'] = load_time

load_time = time.time()
collection_active.add(
    documents=list(data_active['text']),
    ids=[str(i) for i in data_active['id']]
)
load_time = time.time() - load_time
metrics['default']['active']['load_time'] = load_time

load_time = time.time()
collection_hot.add(
    documents=list(data_hot['text']),
    ids=[str(i) for i in data_hot['id']]
)
load_time = time.time() - load_time
metrics['default']['hot']['load_time'] = load_time

In [20]:
search_time = time.time()
queries = search_data_index_vote['query name'].unique()
results_vote = collection_vote.query(
    query_texts=queries, # Chroma will embed this for you
    n_results=40 # how many results to return
)
search_time = time.time() - search_time
metrics['default']['vote']['search_time'] = search_time

search_time = time.time()
queries = search_data_index_hot['query name'].unique()
results_hot = collection_hot.query(
    query_texts=queries, # Chroma will embed this for you
    n_results=40 # how many results to return
)
search_time = time.time() - search_time
metrics['default']['hot']['search_time'] = search_time

search_time = time.time()
queries = search_data_index_active['query name'].unique()
results_active = collection_active.query(
    query_texts=queries, # Chroma will embed this for you
    n_results=40 # how many results to return
)
search_time = time.time() - search_time
metrics['default']['active']['search_time'] = search_time

client.delete_collection("seach_db_vote")
client.delete_collection("seach_db_active")
client.delete_collection("seach_db_hot")

In [21]:
all_results_active.append(results_active)
all_results_hot.append(results_hot)
all_results_vote.append(results_vote)

In [22]:
metrics['default']['vote']['metrics'] = evaluate_algs(results_vote,search_data_index_vote)
metrics['default']['active']['metrics'] = evaluate_algs(results_active,search_data_index_active)
metrics['default']['hot']['metrics'] = evaluate_algs(results_hot,search_data_index_hot)

In [17]:
import torch
from typing import Optional, cast

import numpy as np
import numpy.typing as npt
from chromadb.api.types import EmbeddingFunction, Documents, Embeddings


class TransformerEmbeddingFunction(EmbeddingFunction[Documents]):
    def __init__(
            self, model_name
    ):
        self.model_name=model_name
    def __call__(self, input: Documents) -> Embeddings:

        return run_gte_model(text=input,model_path=model_name).tolist()

    def preprocess_query(self,text_list):
        return text_list

class TransformerEmbeddingFunction_SFR(EmbeddingFunction[Documents]):
    def __init__(
            self
    ):
        return
    def __call__(self, input: Documents) -> Embeddings:

        return sfr_model_inference(text=input).tolist()
    
    def preprocess_query(self,text_list):
        text_list = [ f"Instruct: Given a web search query,\
                      retrieve relevant passages that answer the query\nQuery: {text}"
            for text in text_list]
        return text_list

In [18]:
models_list = ["Mihaiii/gte-micro",
               "thenlper/gte-base",
                "thenlper/gte-large","SFR"]
# model_name="thenlper/gte-large"
# model_embed = TransformerEmbeddingFunction(model_name)
model_name="SFR"
model_embed = TransformerEmbeddingFunction_SFR()

In [73]:
model_name

'thenlper/gte-large'

In [74]:
load_time = 0
search_time = 0

In [75]:
# client.delete_collection("seach_db_vote")
# client.delete_collection("seach_db_active")
# client.delete_collection("seach_db_hot")

In [15]:
import chromadb
client = chromadb.Client()

In [22]:
import chromadb
import time

metrics[model_name] = {"active":{},"vote":{},"hot":{}}

client = chromadb.Client()

collection_vote = client.create_collection("seach_db_vote",get_or_create=False,embedding_function=model_embed)
collection_active = client.create_collection("seach_db_active",get_or_create=False,embedding_function=model_embed)
collection_hot = client.create_collection("seach_db_hot",get_or_create=False,embedding_function=model_embed)
# collection = client.create_collection("test_seach_fb",embedding_function,get_or_create=True)

# load
load_time = time.time()
collection_vote.add(
    documents=list(data_vote['text']),
    ids=[str(i) for i in data_vote['id']]
)
load_time = time.time() - load_time
metrics[model_name]['vote']['load_time'] = load_time

load_time = time.time()
collection_active.add(
    documents=list(data_active['text']),
    ids=[str(i) for i in data_active['id']]
)
load_time = time.time() - load_time
metrics[model_name]['active']['load_time'] = load_time

load_time = time.time()
collection_hot.add(
    documents=list(data_hot['text']),
    ids=[str(i) for i in data_hot['id']]
)
load_time = time.time() - load_time
metrics[model_name]['hot']['load_time'] = load_time

OutOfMemoryError: 

In [ ]:
search_time = time.time()
queries = list(search_data_index_vote['query name'].unique())
queries = model_embed.preprocess_query(queries)
results_vote = collection_vote.query(
    query_texts=queries, # Chroma will embed this for you
    n_results=40 # how many results to return
)
search_time = time.time() - search_time
metrics[model_name]['vote']['search_time'] = search_time

search_time = time.time()
queries = list(search_data_index_hot['query name'].unique())
queries = model_embed.preprocess_query(queries)
results_hot = collection_hot.query(
    query_texts=queries, # Chroma will embed this for you
    n_results=40 # how many results to return
)
search_time = time.time() - search_time
metrics[model_name]['hot']['search_time'] = search_time

search_time = time.time()
queries = list(search_data_index_active['query name'].unique())
queries = model_embed.preprocess_query(queries)
results_active = collection_active.query(
    query_texts=queries, # Chroma will embed this for you
    n_results=40 # how many results to return
)
search_time = time.time() - search_time
metrics[model_name]['active']['search_time'] = search_time

client.delete_collection("seach_db_vote")
client.delete_collection("seach_db_active")
client.delete_collection("seach_db_hot")

## Evaluation

Evaluation
https://www.cs.cmu.edu/~ytsvetko/papers/qvec.pdf